In [1]:
import os
import sys
import json
from collections import defaultdict
import pandas as pd
from IPython.display import display, HTML
project_root = os.path.abspath("..")
sys.path.append(project_root)

# Import metric evaluation from your existing codebase
from evaluation.explanation_evaluation_calc import evaluate_all_cot

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

c:\Users\rishi\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rishi\.cache\huggingface\hub\models--roberta-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [2]:
# Define path structure
datasets = ["truthfulqa", "strategyqa", "medqa", "commonsenseqa"]
models = ["mistral", "llama", "qwen"]
metric_keys = ["redundancy", "weak_relevance", "strong_relevance"]
base_path = os.path.join(project_root, "results", "generation")

In [3]:
# Create master dictionary to collect average scores
summary = defaultdict(dict)

for dataset in datasets:
    for model in models:
        subfolder = f"{dataset}_{model}"
        folder_path = os.path.join(base_path, subfolder)
        
        if not os.path.exists(folder_path):
            print(f"[SKIPPED] Missing folder: {folder_path}")
            continue

        jsonl_files = [f for f in os.listdir(folder_path) if f.endswith(".jsonl")]
        if not jsonl_files:
            print(f"[SKIPPED] No .jsonl file in {folder_path}")
            continue

        filepath = os.path.join(folder_path, jsonl_files[0])
        print(f"[✓] Evaluating: {filepath}")

        try:
            results = evaluate_all_cot(filepath)
        except Exception as e:
            print(f"[ERROR] Failed to evaluate: {filepath}")
            print(e)
            continue

        for metric in metric_keys:
            try:
                avg_score = sum(entry[metric] for entry in results) / len(results)
                summary[dataset][f"{model}_{metric}"] = round(avg_score, 4)
            except Exception as e:
                summary[dataset][f"{model}_{metric}"] = "N/A"


[✓] Evaluating: c:\Users\rishi\Desktop\ImperialMSc\Individual_projec\Code\ExplainabilityInLLMs-MScThesis\results\generation\truthfulqa_mistral\cot_outputs_ollama_meta_reasoning_conclusion_step_indices_fewshot.jsonl


KeyboardInterrupt: 

In [28]:
# Convert summary into DataFrame
df = pd.DataFrame.from_dict(summary, orient="index").reset_index()
df.rename(columns={"index": "Dataset"}, inplace=True)

# Capitalize dataset names
dataset_name_map = {
    "truthfulqa": "TruthfulQA",
    "strategyqa": "StrategyQA",
    "medqa": "MedQA",
    "commonsenseqa": "CommonSenseQA"
}
df["Dataset"] = df["Dataset"].map(dataset_name_map)

# Extract column data and build MultiIndex columns
column_tuples = []
new_data = {}

for model in models:
    pretty_model = model.capitalize() if model != "llama" else "LLaMA"
    for metric in metric_keys:
        pretty_metric = metric.replace("_", " ").title()
        flat_col = f"{model}_{metric}"
        multi_col = (pretty_model, pretty_metric)
        column_tuples.append(multi_col)
        new_data[multi_col] = df[flat_col]

# Create MultiIndex DataFrame
multi_df = pd.DataFrame(new_data)
multi_df.insert(0, ("", "Dataset"), df["Dataset"])  # Insert Dataset as top-level blank
multi_df.columns = pd.MultiIndex.from_tuples(multi_df.columns)

# Overwrite df with multi_df
df = multi_df


In [32]:
# Display styled multi-index table with centered, padded subheaders
custom_table_html = f"""
<style>
th.col_heading.level0 {{
    text-align: center !important;
    padding: 10px 12px;
    background-color: #f9f9f9;
    font-weight: bold;
}}

th.col_heading.level1 {{
    text-align: center !important;
    padding: 8px 16px; /* Increased left-right padding */
}}

td {{
    text-align: center;
    padding: 6px 10px;
}}
</style>
<div style="max-height: 500px; overflow: auto;">
{df.to_html(index=False, escape=False, border=0)}
</div>
"""

display(HTML(custom_table_html))


In [30]:
output_csv = os.path.join(project_root, "results", "cot_method", "cot_metrics_summary.csv")
output_html = os.path.join(project_root, "results", "cot_method", "cot_metrics_summary.html")

# Save as CSV
df.to_csv(output_csv, index=False)
print(f"[✓] CSV saved to: {output_csv}")

# Save as HTML
df.to_html(output_html, index=False)
print(f"[✓] HTML saved to: {output_html}")


[✓] CSV saved to: c:\Users\rishi\Desktop\ImperialMSc\Individual_projec\Code\ExplainabilityInLLMs-MScThesis\results\cot_method\cot_metrics_summary.csv
[✓] HTML saved to: c:\Users\rishi\Desktop\ImperialMSc\Individual_projec\Code\ExplainabilityInLLMs-MScThesis\results\cot_method\cot_metrics_summary.html
